In [ ]:
#REMOVING NULL GENOTYPE VARIANTS
## MENDELIAN INHERITANCE VARIANTS
## WE NEED TO CHECK FOR JUST INHERITED VARIANTS, MEANING PARENTS MUST BE HETEROZYGOUS AND THE ALLELES THE KID HAS MUST BE FROM PARENTS
import re

## we are trying to keep all variants that can be inherited in autosomal recessive fashion(parents heterozygous or homozygous ref) as well as including the possibility of de novo variants
with open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/snpeff_canon.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.merged.vcf', 'rt') as vcf, open("/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/no_null.snpeff_canon.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.merged.vcf", "w") as output:
    
    for line in vcf:
        if line.startswith("#"):
            output.write(line)
        else:
            regex = "GT:\S*?\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)"
            genotype = re.search(regex, line)
            ## remove all variants with no information on child
            if "." in genotype[5] or "." in genotype[6] or "." in genotype[1] or "." in genotype[2] or "." in genotype[3] or "." in genotype[4]:
                pass
            else:
                output.write(line)
                


In [ ]:
#10-03-2023: DE NOVO VARIANTS
## here, we will filter for the de novo variants, dominant negative
import re
with open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/no_null.snpeff_canon.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.merged.vcf', 'rt') as vcf, open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/denovo.no_null.snpeff_canon.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.merged.vcf', 'w') as output:
    regex = "GT:\S*?\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)"

    for line in vcf:
        if line.startswith('#'):
            output.write(line)
        else:
            genotype = re.search(regex, line)
            parent_alleles = [genotype[1], genotype[2], genotype[3], genotype[4]]
            if genotype[5] != '0':
                if genotype[5] not in parent_alleles:
                    output.write(line)
            elif genotype[6] != '0':
                if genotype[6] not in parent_alleles:
                    output.write(line)


        


In [ ]:
### okay, checking the deseq2 results for any effect on the rna. use this on denovo genes

## compound heterozygous; filter for < -1 log2fc
with open("/Users/johnlee/jupyter/new_case/case_study_trio/sourcefiles/C4RCD_0637-VS-UNAFFECTED.DESeq2_results.txt.deseq.vcf", 'rt') as vcf, open("/Users/johnlee/jupyter/new_case/case_study_trio/comp_het.gene_list.txt", "rt") as gene_list, open('comp_het.genes.vcf', 'w') as output:
    genes = []
    for line in gene_list:
        genes.append(line.strip())
    for line in vcf:
        if line.startswith('#'):
            output.write(line)
        else:
            regex = "GENE=(.*?)[;]\S*[;]LOG2FC=(.*?)[;](.*?);"
            gene_name = re.search(regex, line)
            if gene_name[2] != "NA":                
                if gene_name[1] in genes and float(gene_name[2]) <= -1:
                    output.write(line)


In [ ]:
# AUTOSOMAL RECESSIVE VARIANTS

import re
import pandas as pd
with open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/no_null.snpeff_canon.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.merged.vcf', 'rt') as vcf, open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/aut_rec.no_null.snpeff_canon.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.merged.vcf', 'w') as output:
    regex = "GT:\S*?\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)"
    gene_list_regex = "ANN=\S*?[|]\S*?[|]\S*?[|](.*?)[|]"
    gene_list = []
## GENE LIST GENERATION
    for line in vcf:
        if not line.startswith('#'):
            gene_name = re.search(gene_list_regex, line)
            gene_list.append(gene_name[1])
    
    genes_df = pd.DataFrame(gene_list)
    gene_counts = genes_df[0].value_counts().reset_index()
    gene_counts.columns = ['gene', 'count']
    two_hits = gene_counts.loc[gene_counts['count'] >1]

    vcf.seek(0)
    for line in vcf:
        if line.startswith('#'):
            output.write(line)
        else:
            genotype = re.search(regex, line)
            if genotype[5] == genotype[6] and float(genotype[5]) != 0:
                parent1 = [genotype[1], genotype[2]]
                parent2 = [genotype[3], genotype[4]]
                if parent1[0] != parent1[1] and parent2[0] != parent2[1]:

                            
                    if genotype[5] in parent1 and genotype[5] in parent2:
                        output.write(line)    

In [ ]:
## no autosomal recessive...
import re

with open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/no_null.snpeff_canon.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.merged.vcf', 'rt') as vcf, open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/aut_rec.no_null.snpeff_canon.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.merged.vcf', 'w') as output:
    regex = "GT:\S*?\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)"
    gene_list_regex = "ANN=\S*?[|]\S*?[|]\S*?[|](.*?)[|]"

## PARSING THROUGH THE VCF
    for line in vcf:
        if line.startswith('#'):
            output.write(line)
        else:
            genotype = re.search(regex, line)
            if genotype[5] == genotype[6] and float(genotype[5]) != 0:
                if parent1[0] != parent1[1] and parent2[0] != parent2[1]:
                    parent1 = [genotype[1], genotype[2]]
                    parent2 = [genotype[3], genotype[4]]
                            
                    if genotype[5] in parent1 and genotype[5] in parent2:
                        output.write(line)

## no autosomal recessive variants it seems

In [ ]:
# COMPOUND HETEROZYGOUS VARIANTS. modify code to look at the de novo variant genes
import re 
import pandas as pd

##creating both aut_rec and comp_het:

with open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/denovo.no_null.snpeff_canon.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.merged.vcf', 'rt') as vcf:
    regex = "GT:\S*?\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)\S+\s+\S+\s+\S+\s+(.+?)[\/|](.+?)"
    gene_list_regex = "ANN=\S*?[|]\S*?[|]\S*?[|](.*?)[|]"
    gene_list = []
## first make a gene list 
    for line in vcf:
        if not line.startswith('#'):
            gene_name = re.search(gene_list_regex, line)
            if gene_name[1].find('-') > -1 or gene_name[1].find('.') > -1 or gene_name[1].startswith('MUC'):
                pass
            else:
                gene_list.append(gene_name[1])
                
    
    genes_df = pd.DataFrame(gene_list)
    gene_counts = genes_df[0].value_counts().reset_index()
    gene_counts.columns = ['gene', 'count']


In [ ]:
##write out comp het genes to file. do this for de novo too
with open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/denovo.gene_list.txt', 'w') as output:
    for x in gene_counts['gene']:
        if x.find("-") > -1 or x.find('.') > -1 or x.startswith('MUC'):
            pass
        else:
            output.write(x+ "\n")

In [ ]:
## denovo deseq2 results

with open("/Users/johnlee/jupyter/new_case/case_study_trio/sourcefiles/C4RCD_0637-VS-UNAFFECTED.DESeq2_results.txt.deseq.vcf", 'rt') as vcf, open("/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/denovo.gene_list.txt", "rt") as gene_list, open('/Users/johnlee/jupyter/new_case/case_study_trio/new_vcfs/denovo.genes.vcf', 'w') as output:
    genes = []
    for line in gene_list:
        genes.append(line.strip())
    for line in vcf:
        if line.startswith('#'):
            output.write(line)
        else:
            regex = "GENE=(.*?)[;]\S*[;]LOG2FC=(.*?)[;](.*?);"
            gene_name = re.search(regex, line)
            if gene_name[2] != "NA":

                if gene_name[1] in genes and float(gene_name[2]) <= -1:
                    output.write(line)

In [20]:
with open("/Users/johnlee/jupyter/forjohn2/vcfs/reportable_range.dbsnp.no_null.high_mod.qual100.C4RCD_0637_1_PB_Whole_F0020G2S2M0635P0636_TSE61-C4RCD_0635_1_PB_Whole_F0020G2S1MP_TSE61-C4RCD_0636_1_PB_Whole_F0020G1S1MP_TSE61.HC_All.snpEff.vcf", 'rt') as vcf:
    tmp = []
    for line in vcf:
        tmp.append(line)
        

In [22]:

import re 
regex = "^(\S+)\s+(\S+)\s+\S+\s+(\S+)\s+(\S+)\s+\S+\s+\S+\s+(\S+)"
search = re.search(regex, tmp[400])

search

<re.Match object; span=(0, 612), match='1\t12854068\trs79492089;rs75216843\tT\tC\t727.13\>

In [17]:
tmp[58010]

'ERCC-00171\t1\t.\tN\t<EXP>\tNA\t.\tIMPRECISE;END=505;GENEID=ERCC-00171;GENE=ERCC-00171;BASEMEAN=0;LOG2FC=NA;l2fcSE=NA;Stat=NA;PVALUE=NA;PADJ=NA;REFERENCE=C4RCD_0093_2_PB_Whole_F0195G1S1M0132P0133_TSMRU.htSeqCounts\n'